In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [17]:
df = pd.read_csv(r"C:\Users\bless\OneDrive\Desktop\week 5\credit-risk-model\data\raw\data.csv") 

In [18]:
df['TransactionStartTime'] = pd.to_datetime(df['TransactionStartTime'], errors='coerce')
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0


In [19]:
df['transaction_hour'] = df['TransactionStartTime'].dt.hour
df['transaction_day'] = df['TransactionStartTime'].dt.day
df['transaction_month'] = df['TransactionStartTime'].dt.month
df['transaction_weekday'] = df['TransactionStartTime'].dt.weekday
df.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_weekday
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,2,15,11,3
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0,2,15,11,3
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0,2,15,11,3
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,3,15,11,3
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0,3,15,11,3


In [20]:
agg_df = df.groupby('CustomerId').agg(
    total_amount=('Amount', 'sum'),
    avg_amount=('Amount', 'mean'),
    std_amount=('Amount', 'std'),
    transaction_count=('TransactionId', 'count'),
    fraud_count=('FraudResult', 'sum')
).fillna(0).reset_index()

agg_df.head()

,CustomerId,total_amount,avg_amount,std_amount,transaction_count,fraud_count
0,CustomerId_1,-10000.0,-10000.000000,0.000000,1,0
1,CustomerId_10,-10000.0,-10000.000000,0.000000,1,0
2,CustomerId_1001,20000.0,4000.000000,6558.963333,5,0
3,CustomerId_1002,4225.0,384.090909,560.498966,11,0
4,CustomerId_1003,20000.0,3333.333333,6030.478146,6,0


In [21]:
cat_cols = ['ProductCategory', 'ChannelId', 'CurrencyCode']
latest_cats = df.sort_values('TransactionStartTime').groupby('CustomerId')[cat_cols].last().reset_index()
final_df = pd.merge(agg_df, latest_cats, on='CustomerId', how='left')
final_df.head()

,CustomerId,total_amount,avg_amount,std_amount,transaction_count,fraud_count,ProductCategory,ChannelId,CurrencyCode
0,CustomerId_1,-10000.0,-10000.000000,0.000000,1,0,airtime,ChannelId_2,UGX
1,CustomerId_10,-10000.0,-10000.000000,0.000000,1,0,airtime,ChannelId_2,UGX
2,CustomerId_1001,20000.0,4000.000000,6558.963333,5,0,airtime,ChannelId_3,UGX
3,CustomerId_1002,4225.0,384.090909,560.498966,11,0,financial_services,ChannelId_2,UGX
4,CustomerId_1003,20000.0,3333.333333,6030.478146,6,0,airtime,ChannelId_3,UGX


In [22]:
num_cols = ['total_amount', 'avg_amount', 'std_amount', 'transaction_count', 'fraud_count']
cat_cols = ['ProductCategory', 'ChannelId', 'CurrencyCode']

num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, num_cols),
        ('cat', cat_pipeline, cat_cols)
    ]
)

X = final_df.drop(columns=['CustomerId'])  
X_preprocessed = preprocessor.fit_transform(X)
X_preprocessed.shape


(3742, 18)

In [23]:
cat_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(cat_cols)
num_feature_names = num_cols
all_feature_names = np.concatenate([num_feature_names, cat_feature_names])

pd.DataFrame(X_preprocessed.toarray() if hasattr(X_preprocessed, 'toarray') else X_preprocessed, 
             columns=all_feature_names).head()

,total_amount,avg_amount,std_amount,transaction_count,fraud_count,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_1,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5,CurrencyCode_UGX
0,-0.066891,-0.153364,-0.140432,-0.253459,-0.066617,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-0.066891,-0.153364,-0.140432,-0.253459,-0.066617,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,-0.055849,-0.069870,-0.072731,-0.212186,-0.066617,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,-0.061655,-0.091435,-0.134647,-0.150278,-0.066617,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,-0.055849,-0.073846,-0.078186,-0.201868,-0.066617,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [24]:
final_df.to_csv('../data/processed/final_features.csv', index=False)